<figure>
  <IMG SRC="https://raw.githubusercontent.com/mbakker7/exploratory_computing_with_python/master/tudelft_logo.png" WIDTH=200 ALIGN="right">
</figure>

### InSAR data model based on xarray(/dask)

**Steps:**
- Load a raw interferogram (complex(Re, Im)) in binary format into a `xarray.Dataset` object
- Visualize the phase
- Load raw coherence into a `xarray.Dataset` object

In [ ]:
import sarxarray
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from tqdm import tqdm
from skimage.util import view_as_windows
import xarray as xr

**Specify path of file location**

In [ ]:
path = Path('data/')  # CHANGE to local data directory

### Interferogram

**List the interferograms (.raw files) to be read**

In [ ]:
f_ifg = 'cint_srd.raw'  # string

list_ifgs = [p/f_ifg for p in path.rglob("????????")]
list_ifgs

In [ ]:
# Create list with dates
# Mother = 20180108

date_list = []
for i in range(len(list_ifgs)):
    prep_date_string = str(list_ifgs[i])
    date = prep_date_string.split('\\')[3]
    date_list.append(date)
date_list

In [ ]:
# Take the mother-mother ifg out

mother_str = '20180108'
mother_idx = date_list.index(mother_str)

list_ifgs_without_mother = list_ifgs[0:mother_idx]+list_ifgs[(mother_idx+1):10]
list_ifgs_without_mother

**Metadata**

Information about the shape can be found in the ifgs.res files and are denoted using 'nlines' and 'npixels', respectively.

In [ ]:
# Metadata

shape=(5500, 1800)  # obtained from ifgs.res --> nlines = rows ; npixels = columns
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

**Loading the raw interferogram into a `xarray.Dataset`**

In [ ]:
# Create xarray.Dataset object from .raw file

ifg_stack = sarxarray.from_binary(list_ifgs_without_mother, shape, dtype=dtype)
ifg_stack = ifg_stack.chunk({"azimuth":500, "range":500, "time":1 })  # set custom chunk sizes

ifg_stack

In [ ]:
phase = ifg_stack.phase
amplitude = ifg_stack.amplitude
phasor = ifg_stack.complex # contains P00, P01, P02

In [ ]:
phase

**Visualize the phase**

In [ ]:
# Visualize first figure

fig,ax = plt.subplots(1,1)
phase_i = phase.isel(time=1)
ax.imshow(phase_i)
phase_i.plot(robust=True, ax=ax, cmap='jet')  # cmap='jet'

In [ ]:
# Visualize 3/9 figures

phase_ex = ifg_stack.phase.isel(time=slice(3,6))  
phase_ex.plot(x="range", y="azimuth", col="time", col_wrap=3, cmap='jet')

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(25, 25), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .25, wspace=.1)

axs = axs.ravel()

for i in tqdm(range(len(ifg_stack.time))):
    phase_i = phase.isel(time=i)
    axs[i].imshow(phase_i)
    phase_i.plot(robust=True, ax=axs[i], cmap='jet')  # cmap='jet'


**MRM (Mean Reflection Map)**

In [ ]:
# Creating a MRM (Mean Reflection Map) of a subset of the stack

mrm = ifg_stack.slcstack.mrm() # go 3D to 2D --> only azimuth & range for amplitude
mrm

In [ ]:
mrm_subset = mrm[1000:2000, 500:2000]  # Create subset using 2 indexes: azimuth & range

mrm_subset = mrm_subset.compute() # manually trigger loading of this array’s data from disk or a remote source into memory and return a new array
mrm_subset

In [ ]:
# Visualize

fig, ax = plt.subplots()
ax.imshow(mrm_subset)
mrm_subset.plot(robust=True, ax=ax)

**Load raw coherence**

In [ ]:
f_coh = 'coherence.raw'  # string

list_coh = [p/f_coh for p in path.rglob("????????")]
list_coh = list_coh[0:mother_idx] + list_coh[mother_idx+1:10]  # do not include coherence of mother-mother ifg

In [ ]:
# Metadata

shape=(275, 450)  # obtained from ifgs.res --> nlines = rows ; npixels = columns

# Create xarray.Dataset object from .raw file

coh_stack = sarxarray.from_binary(list_coh, shape, dtype=np.float32)
coh_stack = coh_stack.chunk({"azimuth":100, "range":100, "time":1 })  # set custom chunk sizes
coh_stack

In [ ]:
# Visualize coherence

fig,ax = plt.subplots(1,1)

for i in tqdm(range(len(coh_stack.time))):
    coh_i = coh_stack.amplitude.isel(time=i)
    plt.imshow(coh_i)
    coh_i.plot(robust=True, cmap='bone',vmax=0.7,vmin=0)  # cmap='jet'
    plt.show() 

**Load slave_rsmp - to get original amplitude of SLC's e.g.**

In [ ]:
path_mother = Path('data_mother')  
f_mother = 'slave_rsmp.raw'  # Load complex data of mother to obtain amplitude

shape=(5500, 1800)  # obtained from ifgs.res --> nlines = rows ; npixels = columns
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

mother = [p/f_mother for p in path_mother.rglob("????????")]

mother = sarxarray.from_binary(mother, shape, dtype=dtype)
mother = mother.chunk({"azimuth":500, "range":500, "time":1 })  # set custom chunk sizes
mother.amplitude

**Multi-looking**

In [ ]:
def multilooking(data, window_size, variable_name):
    
    # Generate patches
    
    patches_real = view_as_windows(np.real(data), window_size, step=window_size)  # step is important as its value can result in overlapping or non overlapping patches
    
    # Compute the mean of each patch
    
    real_mean = np.nanmean(patches_real, axis=(2, 3))  # the 3rd and 4th axes represent the window dimensions
    
    # Consider the imaginary part; in the case input data is a complex number
    
    if not np.all(np.imag(data) == 0):  # if imaginary
        
        patches_imag = view_as_windows(np.imag(data), window_size, step=window_size)
        
        # Compute the mean of each patch
        
        imag_mean = np.nanmean(patches_imag, axis=(2, 3))
        
        # Combine the real and imaginary part
        
        output_array = real_mean + 1j * imag_mean
        
        # Save as xarray dataset
        
        comp = xr.DataArray(output_array, dims=None)
        ph = xr.DataArray(np.angle(output_array), dims=('azimuth','range'))
        amp = xr.DataArray(np.abs(output_array), dims=('azimuth','range'))
        
        output_array = xr.DataArray(comp, 
                        coords={'azimuth': np.arange(0, np.shape(output_array)[0], 1, dtype=int),
                        'range': np.arange(0, np.shape(output_array)[1], 1, dtype=int)}, 
                        dims=["azimuth","range"])
        output_array= output_array.to_dataset(name='complex')

        output_array['amplitude'] = amp
        output_array['phase'] = ph
        
    else:
        
        output_array = real_mean
        
        # Save as xarray dataset
        
        output_array = xr.DataArray(output_array, 
                        coords={'azimuth': np.arange(0, np.shape(output_array)[0], 1, dtype=int),
                        'range': np.arange(0, np.shape(output_array)[1], 1, dtype=int)}, 
                        dims=["azimuth","range"])
        output_array = output_array.to_dataset(name=variable_name)
        
    return output_array

In [ ]:
# Apply multilooking

first_skip = False
count = 0
coords = []

window_size = (22,10)

ifg_ml0 = multilooking(ifg_stack.complex.isel(time=0).values, window_size=window_size, variable_name='complex')

for i in range(len(ifg_stack.time)):
    if(first_skip):
        toAdd_ifg = multilooking(ifg_stack.complex.isel(time=i).values, window_size=window_size, variable_name='complex')
        ifg_ml0 = xr.concat([ifg_ml0, toAdd_ifg], dim="time")
    first_skip = True 
    
    coords.append(count)
    count+=1 
    
ifg_ml = ifg_ml0.assign_coords(time=coords)

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(25, 25), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .25, wspace=.1)

axs = axs.ravel()

for i in tqdm(range(len(ifg_ml.time))):
    axs[i].imshow(ifg_ml.phase.isel(time=i))
    ifg_ml.phase.isel(time=i).plot(robust=True, ax=axs[i], cmap='jet')  # cmap='jet'